<a href="https://colab.research.google.com/github/pedrofuentes79/RNNs/blob/master/Named-Entity-Recognition/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



In [14]:
drive.mount("/content/drive")


# Read the dataset as text
with open("/content/drive/MyDrive/ColabProjects/ner/wikigold/wikigold.conll.txt", 'r', encoding='utf-8') as file:
    dataset_text = file.read()

# Pre-process the dataset to remove trailing "\n" characters
lines = dataset_text.strip().split('\n')
lines = [line.rstrip('\n') for line in lines]

data = []
for line in lines:
    row = line.split(' ')
    data.append(row)

df = pd.DataFrame(data, columns=['Token', 'Entity'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Remove NA values
df = df.dropna()
df = df.reset_index(drop=True)

In [16]:
# check the label types
df["Entity"].value_counts()

# map the labels to numbers
labels = df["Entity"].unique()
label_to_index = {}
index_to_label = {}
for i, label in enumerate(labels):
    label_to_index[label] = i
    index_to_label[i] = label

df["Label"] = df["Entity"].map(label_to_index)

O         32721
I-ORG      1958
I-PER      1634
I-LOC      1447
I-MISC     1392
Name: Entity, dtype: int64


In [17]:
# find all "-DOCSTART-" tokens
docstart_positions = df[df['Token'] == '-DOCSTART-'].index.tolist()
print(docstart_positions)

In [18]:
# create a new column listing the sentence number for each token
sentence_number = 0

for index, row in df.iterrows():
    if index in docstart_positions:
        sentence_number += 1
    df.loc[index, 'Sentence #'] = sentence_number

In [ ]:
# drop all "-DOCSTART-" tokens
df = df[df['Token'] != '-DOCSTART-'].reset_index(drop=True)

In [21]:
# group by sentence number and join tokens into a sentence
df_grouped = df.groupby('Sentence #').agg({'Token': list, 'Label': list}).reset_index()

df_grouped


In [ ]:
### TOKENIZE DATA ###
# create a new column which will have all the words from a sentence. (in this case, they will be word embeddings for each word)